In [31]:
import numpy as np
import tensorflow as tf


ImportError: This module is deprecated.  Use tf.nn.seq2seq instead.

In [1]:
# Neural networks take input as vectors so we have to convert integers to vectors using one-hot encoding
# This function will encode a given integer sequence into RNN compatible format (one-hot representation)

def encode(X,seq_len, vocab_size):
    x = np.zeros((len(X),seq_len, vocab_size), dtype=np.float32)
    for ind,batch in enumerate(X):
        for j, elem in enumerate(batch):
            x[ind, j, elem] = 1
    return x

In [3]:
def batch_gen(batch_size=32, seq_len=10, max_no=100):
    # Randomly generate a batch of integer sequences (X) and its sorted
    # counterpart (Y)
    x = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)
    y = np.zeros((batch_size, seq_len, max_no), dtype=np.float32)

    X = np.random.randint(max_no, size=(batch_size, seq_len))
    Y = np.sort(X, axis=1)
    X=np.eye(max_no)[X]
    Y=np.eye(max_no)[Y]

    return X,Y

In [19]:
seq_len = 10
vocab_len = 100
state_size = 100
num_layers = 4

In [33]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32,[None,seq_len,vocab_len], "in_seq")
y_ = tf.placeholder(tf.float32,[None,seq_len], "in_seq_order")
batch_size = tf.shape(x)[0]

init_state = tf.zeros([batch_size, state_size])

weights = {
    'U': tf.get_variable(name='wU',shape=[seq_len*vocab_len, state_size],initializer=tf.contrib.layers.xavier_initializer()),
    'W': tf.get_variable(name='wW',shape=[state_size, state_size],initializer=tf.contrib.layers.xavier_initializer()),
    'V': tf.get_variable(name='wV',shape=[state_size, seq_len],initializer=tf.contrib.layers.xavier_initializer())
    
}

biases ={
    'U': tf.get_variable(name='bU',shape=[state_size],initializer=tf.contrib.layers.xavier_initializer()),
    'W': tf.get_variable(name='bW',shape=[state_size],initializer=tf.contrib.layers.xavier_initializer()),
    'V': tf.get_variable(name='bV',shape=[seq_len],initializer=tf.contrib.layers.xavier_initializer())
    
}


cell = tf.nn.rnn_cell.LSTMCell(state_size,state_is_tuple=True)
cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers)

val, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
outputs, states = tf.nn.seq2seq(x, decoder_inputs, cell)


NameError: name 'decoder_inputs' is not defined

In [22]:
x = tf.placeholder(tf.int64,[None,seq_len,vocab_len], "in_seq")
batch_size = tf.shape(x)[0]
x_reshaped = tf.reshape(x,[batch_size,seq_len*vocab_len])
b = tf.transpose(x_reshaped)
